In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain

In [2]:
from getpass import getpass
OPENAI_API_KEY = getpass()

········


In [3]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [8]:
template = """Tell me a very short {speech} with 2 sentences."""
prompt = PromptTemplate(template=template, input_variables=["speech"])

In [9]:
llm = OpenAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [10]:
speech = "joke"
output = llm_chain.run(speech)



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a very short joke with 2 sentences.

> Finished chain.


In [11]:
print(output)



Q: Why don't scientists trust atoms?
A: Because they make up everything!


In [249]:
llm = ChatOpenAI(temperature=0.1, max_tokens= 512, model_name='gpt-3.5-turbo')

## Step 1 : Identify Functional Requirements

In [97]:
prompt_template = '''You are an expert in distributed system. 
The scope of your task will be to identify functional requirements from a given system design problem statement.
On response you should list down the functional requirements as you understand.

## Example:
Problem Statement : Design a Chat Application like WhatsApp, where an user can send or receive message, create a group with more than 2 people, gets notification when a new message is received.

## Sample Output:
1. Messaging functionality - allow users to send and receive messages through the chat application.
2. Group creation functionality - enable users to create groups within the application, which they can join to communicate with other members.
3. Notification functionality - notify users when someone sends them a new message, or when a new member joins their group.

{problem_statement}
'''
prompt_fr = PromptTemplate(template=prompt_template, input_variables=["problem_statement"])

In [98]:
llm_chain_fr = LLMChain(prompt=prompt_fr, llm=llm, verbose=True)

In [86]:
problem_statement = '''Design a photo sharing application like instagram, 
where users can post photoes or vides, follow other users, browse throught the photo, video posted by the following users, add comments on those posts.
'''
output = llm_chain_fr.run(problem_statement = problem_statement)
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
You are an expert in distributed system. 
The scope of your task will be to identify functional requirements from a given system design problem statement.
On response you should list down the functional requirements as you understand.

## Example:
Problem Statement : Design a Chat Application like WhatsApp, where an user can send or receive message, create a group with more than 2 people, gets notification when a new message is received.

## Sample Output:
1. Messaging functionality - allow users to send and receive messages through the chat application.
2. Group creation functionality - enable users to create groups within the application, which they can join to communicate with other members.
3. Notification functionality - notify users when someone sends them a new message, or when a new member joins their group.

Design a photo sharing application like instagram, 
where users can post photoes or vides, follow other users, 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).



> Finished chain.
1. Photo and video sharing functionality - allow users to upload and share photos and videos within the application.
2. Following functionality - enable users to follow other users and view their posts in a personalized feed.
3. Browsing functionality - allow users to browse through posts made by the users they follow.
4. Commenting functionality - enable users to add comments on posts made by other users.
5. User profile functionality - allow users to create and manage their own profile, including their posts and followers.


## Step 2 : Identify Components from Problem Statement and Functional Requirements

In [11]:
def custom_json_escape(json_str):
    json_str = json_str.replace('{', '{{')
    json_str = json_str.replace('}', '}}')
    return json_str

json_output_schema = custom_json_escape('{{"$schema": "http://json-schema.org/draft-04/schema#", "type": "array", "items": [{ "type": "object", "properties": { "name": { "type": "string", "description" : "The component name that satsfies a purpose in the system" }, "description": { "type": "string", "description" : "A brief description about the component and its responsibilities and how it solves one or more functional requirement" }, "component_type": { "type": "string", "description" : "Whether the component is a service or storage or Cache or others like load balancer" } }, "required": ["component", "description", "type"]}]}}')

In [12]:
prompt_template_component = '''
You are an assistant to design a distributed System. 
Given a list of functional requirements, you have to provide **high level Components** of the system under design.
1. Each component should be either of the types - Storage, Service, Cache, Load balancer etc. 
2. The Service components should be independent and mutually exclusive of responsibilities.
3. The database component should be present and the requirements should be stated clearly.
Output format should a valid JSON that validates following schema -
{json_output_schema}

{func_requirement}
'''
prompt_component = PromptTemplate(template=prompt_template_component, 
                        input_variables=["json_output_schema","func_requirement"])

In [13]:
llm_chain_comp = LLMChain(prompt=prompt_component, llm=llm, verbose=True)

In [102]:

func_requirement = '''1. Posting functionality - allow users to post photos and videos to their profile.
2. Follow/unfollow functionality - allow users to follow/unfollow other users, and view their posts in their feed.
3. Browsing functionality - allow users to browse through photos and videos posted by the users they follow.
4. Commenting functionality - allow users to add comments on posts they view.'''

output = llm_chain_comp.run(
    json_output_schema = json_output_schema,
    func_requirement = func_requirement
)
print(output)



> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant to design a distributed System. 
Given a list of functional requirements, you have to provide **high level Components** of the system under design.
1. Each component should be either of the types - Storage, Service, Cache, Load balancer etc. 
2. The Service components should be independent and mutually exclusive of responsibilities.
3. The database component should be present and the requirements should be stated clearly.
Output format should a valid JSON that validates following schema -
{{{{"$schema": "http://json-schema.org/draft-04/schema#", "type": "array", "items": [{{ "type": "object", "properties": {{ "name": {{ "type": "string", "description" : "The component name that satsfies a purpose in the system" }}, "description": {{ "type": "string", "description" : "A brief description about the component and its responsibilities and how it solves one or more functional requirement" }}, "component_type":

In [14]:
prompt_template_component_2 = '''
You are an assistant to design a distributed System. 
Given a list of functional requirements, you have to provide **high level Components** of the system under design.
1. Each component should be either of the types - Storage, Service, Cache, Load balancer etc. 
2. The Service components should be independent and mutually exclusive of responsibilities.
3. The database component should be present and the requirements should be stated clearly.
Output format should a valid JSON that validates following schema -
{{{{"$schema": "http://json-schema.org/draft-04/schema#", "type": "array", "items": [{{ "type": "object", "properties": {{ "name": {{ "type": "string", "description" : "The component name that satsfies a purpose in the system" }}, "description": {{ "type": "string", "description" : "A brief description about the component and its responsibilities and how it solves one or more functional requirement" }}, "component_type": {{ "type": "string", "description" : "Whether the component is a service or storage or Cache or others like load balancer" }} }}, "required": ["component", "description", "type"]}}]}}}}

{problem_statement}
'''
prompt_component_2 = PromptTemplate(template=prompt_template_component_2, 
                        input_variables=["problem_statement"])

In [15]:
llm_chain_comp2 = LLMChain(prompt=prompt_component_2, llm=llm, verbose=True)

In [158]:
from langchain.chains import SimpleSequentialChain, SequentialChain

In [106]:
overall_chain = SimpleSequentialChain(chains=[llm_chain_fr, llm_chain_comp2], verbose=True)
final_output = overall_chain.run(problem_statement)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an expert in distributed system. 
The scope of your task will be to identify functional requirements from a given system design problem statement.
On response you should list down the functional requirements as you understand.

## Example:
Problem Statement : Design a Chat Application like WhatsApp, where an user can send or receive message, create a group with more than 2 people, gets notification when a new message is received.

## Sample Output:
1. Messaging functionality - allow users to send and receive messages through the chat application.
2. Group creation functionality - enable users to create groups within the application, which they can join to communicate with other members.
3. Notification functionality - notify users when someone sends them a new message, or when a new member joins their group.

Design a photo sharing application like instagram, 
where users

In [107]:
print(final_output)

[
  {
    "name": "User Service",
    "description": "Responsible for user authentication, registration, and management.",
    "component_type": "Service"
  },
  {
    "name": "Post Service",
    "description": "Responsible for managing posts, including uploading, sharing, and browsing.",
    "component_type": "Service"
  },
  {
    "name": "Follow Service",
    "description": "Responsible for managing user follow relationships.",
    "component_type": "Service"
  },
  {
    "name": "Comment Service",
    "description": "Responsible for managing comments on posts.",
    "component_type": "Service"
  },
  {
    "name": "Media Storage",
    "description": "Responsible for storing photos and videos uploaded by users.",
    "component_type": "Storage"
  },
  {
    "name": "Database",
    "description": "Responsible for storing user, post, and follow data.",
    "component_type": "Storage"
  },
  {
    "name": "Cache",
    "description": "Responsible for caching frequently accessed data to 

In [235]:
llm

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.3, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=512)

## Step 3 :  Independent Component Design

In [267]:
prompt_template_designer = '''
You are an assistant to design a distributed System. 
Given an input about a single component of a large system, you have to design the component.
1. You should understand the requirement of this component from the description
2. Your design should consider and list multiple approaches to solve this particular requirement in very short, compare them based on the use case and suggest the best possible option.
3. On Storage type component, you should identify what type of data to store from the description of the input 
and which database should be appropriate for the data, 
4. if there are different data types, consider different and best storage for each of them.
5. Suggest a well-known cloud offering that satisfies the requirement of the component from {cloud_provider} cloud provider.

You should iterate to each of the points mentioned above to refine the design.

{problem_statement}

'''
prompt_designer = PromptTemplate(template=prompt_template_designer, 
                        input_variables=["problem_statement", "cloud_provider"])

In [274]:
llm_chain_desginer = LLMChain(prompt=prompt_designer, llm=llm, verbose=True)

component = '''{
    "name": "Media Storage",
    "description": "Responsible for storing photos and videos uploaded by users.",
    "component_type": "Storage"
  }'''

c2 = "name='Storage' description='Stores user data, posts, comments, and media files' component_type='Storage'"

output = llm_chain_desginer.run(
    problem_statement = c2,
    cloud_provider = "azure"
)
print(output)



> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant to design a distributed System. 
Given an input about a single component of a large system, you have to design the component.
1. You should understand the requirement of this component from the description
2. Your design should consider and list multiple approaches to solve this particular requirement in very short, compare them based on the use case and suggest the best possible option.
3. On Storage type component, you should identify what type of data to store from the description of the input 
and which database should be appropriate for the data, 
4. if there are different data types, consider different and best storage for each of them.
5. Suggest a well-known cloud offering that satisfies the requirement of the component from azure cloud provider.

You should iterate to each of the points mentioned above to refine the design.

name='Storage' description='Stores user data, posts, comments, and media

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).



> Finished chain.
1. Requirement: The component should be able to store different types of data such as user data, posts, comments, and media files.

2. Approaches:
- Relational database: This approach involves using a traditional relational database such as MySQL or PostgreSQL to store the data. This approach is suitable for structured data and provides strong consistency and ACID transactions. However, it may not be the best option for storing unstructured data such as media files.
- NoSQL database: This approach involves using a NoSQL database such as MongoDB or Cassandra to store the data. This approach is suitable for unstructured data and provides high scalability and availability. However, it may not provide strong consistency and ACID transactions.
- Object storage: This approach involves using an object storage service such as Amazon S3 or Azure Blob Storage to store the data. This approach is suitable for storing large unstructured data such as media files and provides high 

In [315]:
class ServiceComponent(BaseModel):
    '''Represents a Service Component with a design'''
    requirement: str = Field(description="Requirements of the service component")
    apis: List[str] = Field(description="apis of the service")
    conclusion: str = Field(description="overall conclusions on the component")

    def __str__(self) -> str:
        apistr = '\t'
        for api in self.apis:
            apistr += api + '\n\t'
        return f'''#### Requirements:\n{self.requirement}\n#### APIs\n{apistr}\n#### Comments\n{self.conclusion}\n
        '''

In [317]:
prompt_template_service_designer = '''
You are an assistant to design a distributed System. 
Given an input about a single service component of a large system, you have to design the component.
1. You should understand the requirement of this component from the description.
2. You should provide the minimum list of APIs that this component should handle. 

You should iterate to each of the points mentioned above to refine the design.
Your design should be very specific to the problem and do not generate extra outputs. 

{format_instructions}

## Example
{{
    "name" : "Order Service",
    "description" : "Responsible for placing an order.",
    "component_type" : "Service"
}}

## Sample Output:

{{
    "requirement": "Responsible for checking out one or more items in a cart and places an order",
    "apis": ["AddToCart(item) - Adds one item to the cart", "DeleteFromCart(item) - deletes one item from the cart.", "CreateOrder(orderId) - Create an order from the cart items.", "PlaceOrder(orderId) - Initiates the payment for the order"],
    "conclusion": "With the apis, the ordering service fulfills the minimal requirements of placing an order."
}}


Now design the service component for the below -
{problem_statement}

'''
parser2 = PydanticOutputParser(pydantic_object=ServiceComponent)
prompt_designer_svc = PromptTemplate(template=prompt_template_service_designer, 
                        input_variables=["problem_statement"],
                        partial_variables={"format_instructions": parser2.get_format_instructions()})

llm_chain_desginer_svc = LLMChain(prompt=prompt_designer_svc, llm=llm, verbose=True)

In [318]:
component = '''{
    "name": "Comment Service",
    "description": "Responsible for managing comments on posts.",
    "component_type": "Service"
  }'''

output = llm_chain_desginer_svc.run(
    problem_statement = component
)
print(output)



> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant to design a distributed System. 
Given an input about a single service component of a large system, you have to design the component.
1. You should understand the requirement of this component from the description.
2. You should provide the minimum list of APIs that this component should handle. 

You should iterate to each of the points mentioned above to refine the design.
Your design should be very specific to the problem and do not generate extra outputs. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```


In [319]:
comp_des = parser2.parse(output)
print(comp_des)

#### Requirements:
Responsible for managing comments on posts
#### APIs
	CreateComment(postId, comment) - Creates a new comment on the post with the given postId
	GetComments(postId) - Retrieves all comments on the post with the given postId
	UpdateComment(commentId, comment) - Updates the comment with the given commentId
	DeleteComment(commentId) - Deletes the comment with the given commentId
	
#### Comments
With the apis, the comment service fulfills the minimal requirements of managing comments on posts.

        


In [279]:
prompt_template_misc_designer = '''
You are an assistant to design a distributed System. 
Given an input about a component of a large system, you have to design the component.
1. You should understand the requirement of this component from the description.
2. You should enlist a few approaches to solve the problem and the requirement.
3. Suggest a well-known cloud offering that satisfies the requirement of the component from {cloud_provider} cloud provider.

You should iterate to each of the points mentioned above to refine the design.
Your design should be very specific to the problem and do not generate extra outputs.

{problem_statement}

'''
prompt_designer_misc = PromptTemplate(template=prompt_template_misc_designer, 
                        input_variables=["problem_statement", "cloud_provider"])



llm_chain_desginer_misc = LLMChain(prompt=prompt_designer_misc, llm=llm, verbose=True)

In [281]:
ps = '''{
    "name": "Load Balancer",
    "description": "Responsible for distributing incoming requests across multiple instances of the service components.",
    "component_type": "Load balancer"
  }'''

output = llm_chain_desginer_misc.run(problem_statement = ps, cloud_provider='Azure')



> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant to design a distributed System. 
Given an input about a component of a large system, you have to design the component.
1. You should understand the requirement of this component from the description.
2. You should enlist a few approaches to solve the problem and the requirement.
3. Suggest a well-known cloud offering that satisfies the requirement of the component from Azure cloud provider.

You should iterate to each of the points mentioned above to refine the design.
Your design should be very specific to the problem and do not generate extra outputs.

{
    "name": "Load Balancer",
    "description": "Responsible for distributing incoming requests across multiple instances of the service components.",
    "component_type": "Load balancer"
  }



> Finished chain.


In [282]:
print(output)

1. Requirement: The Load Balancer component should be able to distribute incoming requests across multiple instances of the service components.

2. Approaches:
- Round-robin approach: The Load Balancer distributes incoming requests equally across all available instances of the service components.
- Weighted approach: The Load Balancer distributes incoming requests based on the weight assigned to each instance of the service components. The weight can be based on the capacity of the instance or any other relevant factor.
- Least connections approach: The Load Balancer distributes incoming requests to the instance with the least number of active connections.
- IP hash approach: The Load Balancer distributes incoming requests based on the source IP address of the request. This ensures that requests from the same IP address are always directed to the same instance of the service component.

3. Azure Cloud Offering: Azure Application Gateway is a cloud offering that satisfies the requiremen

## Output Parser

In [4]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [140]:
class Component(BaseModel):
    name: str = Field(description="name of the component")
    description: str = Field(description="description of the component")
    component_type: str = Field(description="type of the component")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('component_type')
    def component_type_validation(cls, field):
        if field not in ['Storage', 'Service', 'Load Balancer', 'Cache', 'Database']:
            raise ValueError(f'not a valid component : {field}')
        return field
    
class Components(BaseModel):
    components: list[Component] = Field(description="List of Components")


In [141]:
parser = PydanticOutputParser(pydantic_object=Components)

In [255]:
prompt_template_parser = '''
You are an assistant to design a distributed System. 
Given a list of functional requirements, you have to provide **high level Components** of the system under design.
1. Each component should be either of the types - Storage, Service, Cache, Load Balancer. 
2. The Service components should be independent and mutually exclusive of responsibilities.
3. The Storage component should be present and the requirements should be stated clearly.
4. Cache component should only be present if there is a necessary of caching of data.
5. Load Balancer component should specify its requirements very specifically.

{format_instructions}

Output based on the functional requirement as below : 
{problem_statement}
'''
prompt_component = PromptTemplate(template=prompt_template_parser, 
                        input_variables=["problem_statement"],
                        partial_variables={"format_instructions": parser.get_format_instructions()}
                    )

In [256]:
func_requirement = '''1. Photo and video sharing functionality - allow users to upload and share photos and videos within the application.
2. Following functionality - enable users to follow other users and view their posts in a personalized feed.
3. Browsing functionality - allow users to browse through posts made by the users they follow.
4. Commenting functionality - enable users to add comments on posts made by other users.
5. User profile functionality - allow users to create and manage their own profile, including their posts and followers.'''

In [257]:
_input = prompt_component.format_prompt(problem_statement=func_requirement)

In [258]:
print(_input.to_string())


You are an assistant to design a distributed System. 
Given a list of functional requirements, you have to provide **high level Components** of the system under design.
1. Each component should be either of the types - Storage, Service, Cache, Load Balancer. 
2. The Service components should be independent and mutually exclusive of responsibilities.
3. The Storage component should be present and the requirements should be stated clearly.
4. Cache component should only be present if there is a necessary of caching of data.
5. Load Balancer component should specify its requirements very specifically.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz

In [259]:
### Use it in a chain

In [260]:
chain_func_req = LLMChain(prompt = prompt_fr, llm=llm, verbose=True)
chain_comp = LLMChain(prompt=prompt_component, llm=llm, verbose=True)

In [261]:
chain_func_req.input_keys, chain_func_req.output_keys

(['problem_statement'], ['text'])

In [262]:
chain_comp.input_keys, chain_comp.output_keys

(['problem_statement'], ['text'])

In [263]:
overall_chain = SimpleSequentialChain(chains=[chain_func_req, chain_comp], verbose=True)

In [264]:
overall_chain.input_keys

['input']

In [265]:
problem_statement = '''Design a photo sharing application like instagram, 
where users can post photoes or vides, follow other users, browse throught the photo, video posted by the following users, add comments on those posts.
'''
final_output = overall_chain.run(problem_statement)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an expert in distributed system. 
The scope of your task will be to identify functional requirements from a given system design problem statement.
On response you should list down the functional requirements as you understand.

## Example:
Problem Statement : Design a Chat Application like WhatsApp, where an user can send or receive message, create a group with more than 2 people, gets notification when a new message is received.

## Sample Output:
1. Messaging functionality - allow users to send and receive messages through the chat application.
2. Group creation functionality - enable users to create groups within the application, which they can join to communicate with other members.
3. Notification functionality - notify users when someone sends them a new message, or when a new member joins their group.

Design a photo sharing application like instagram, 
where users

In [155]:
op = parser.parse(final_output)

In [156]:
op.components

[Component(name='Load Balancer', description='Distributes incoming traffic across multiple service instances', component_type='Load Balancer'),
 Component(name='Service 1', description='Handles posting functionality', component_type='Service'),
 Component(name='Service 2', description='Handles following functionality', component_type='Service'),
 Component(name='Service 3', description='Handles browsing functionality', component_type='Service'),
 Component(name='Service 4', description='Handles commenting functionality', component_type='Service'),
 Component(name='Cache', description='Stores frequently accessed data to reduce database load', component_type='Cache'),
 Component(name='Database', description='Stores all user data and posts', component_type='Storage')]

In [224]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Any, Dict, List, Optional
from langchain.input import get_color_mapping
from pydantic import Extra, root_validator

from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)

from typing import Dict, List

class SequentialOutputChain(Chain):

    chains: List[Chain]
    chained_input_key: str = 'input' #: :meta private:
    output_key: str = 'output'
    
    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set()
        for chain in self.chains:
            all_input_vars = all_input_vars.union(set(chain.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key] + [f'chain_{i}' for i in range(len(self.chains))]

    def _call(self, 
              inputs: Dict[str, str],
              run_manager: Optional[CallbackManagerForChainRun] = None,
        ) -> Dict[str, str]:
        
        output = dict()
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        color_mapping = get_color_mapping([str(i) for i in range(len(self.chains))])
        for i, chain in enumerate(self.chains):
            _output = chain.run(inputs, callbacks=_run_manager.get_child())
            output[f'chain_{i}'] = _output
            inputs[self.chained_input_key] = _output # set as input for next chain
            _run_manager.on_text(
                inputs, color=color_mapping[str(i)], end="\n", verbose=self.verbose
            )
            
        output[self.output_key] = _output
        return output
            

In [225]:
cchain = SequentialOutputChain(chains=[chain_func_req, chain_comp], chained_input_key='problem_statement', verbose=True)

In [226]:
cchain.input_keys, cchain.output_keys

(['problem_statement'], ['output', 'chain_0', 'chain_1'])

In [227]:
ccoutput = cchain(problem_statement, return_only_outputs=True)



> Entering new SequentialOutputChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an expert in distributed system. 
The scope of your task will be to identify functional requirements from a given system design problem statement.
On response you should list down the functional requirements as you understand.

## Example:
Problem Statement : Design a Chat Application like WhatsApp, where an user can send or receive message, create a group with more than 2 people, gets notification when a new message is received.

## Sample Output:
1. Messaging functionality - allow users to send and receive messages through the chat application.
2. Group creation functionality - enable users to create groups within the application, which they can join to communicate with other members.
3. Notification functionality - notify users when someone sends them a new message, or when a new member joins their group.

Design a photo sharing application like instagram, 
where users

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).



> Finished chain.
{'problem_statement': '1. Photo and video posting functionality - allow users to upload and share photos and videos on the application.\n2. Following functionality - enable users to follow other users on the application to view their posts in their feed.\n3. Browsing functionality - allow users to browse through posts made by the users they follow.\n4. Commenting functionality - enable users to add comments on posts made by other users.'}


> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant to design a distributed System. 
Given a list of functional requirements, you have to provide **high level Components** of the system under design.
1. Each component should be either of the types - Storage, Service, Cache, Load Balancer. 
2. The Service components should be independent and mutually exclusive of responsibilities.
3. The database component should be present and the requirements should be stated clearly.

The output should be formatted as

In [228]:
ccoutput.keys()

dict_keys(['chain_0', 'chain_1', 'output'])

In [229]:
ccoutput['chain_0']

'1. Photo and video posting functionality - allow users to upload and share photos and videos on the application.\n2. Following functionality - enable users to follow other users on the application to view their posts in their feed.\n3. Browsing functionality - allow users to browse through posts made by the users they follow.\n4. Commenting functionality - enable users to add comments on posts made by other users.'

In [230]:
components = parser.parse(ccoutput['output'])

In [231]:
components.components

[Component(name='Storage', description='Stores user data, posts, comments, and media files', component_type='Storage'),
 Component(name='User Service', description='Handles user authentication, registration, and profile management', component_type='Service'),
 Component(name='Post Service', description='Handles post creation, retrieval, and deletion', component_type='Service'),
 Component(name='Media Service', description='Handles media file upload, retrieval, and deletion', component_type='Service'),
 Component(name='Follow Service', description='Handles user follow and unfollow functionality', component_type='Service'),
 Component(name='Feed Service', description='Generates user feeds based on the users they follow', component_type='Service'),
 Component(name='Comment Service', description='Handles comment creation, retrieval, and deletion', component_type='Service'),
 Component(name='Cache', description='Caches frequently accessed data to improve performance', component_type='Cache'

In [272]:
str(components.components[0])

"name='Storage' description='Stores user data, posts, comments, and media files' component_type='Storage'"